<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Data Processing Workflow

In this workflow within PyCCAPT, we can crope the data, do the voltage and bowl calibration, calculate the 3d reconstruction, and do the ranging.
</div>

In [ ]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import numpy as np
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import fixed, interact_manual
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.tutorials.tutorials_helpers import helper_calibration
from pyccapt.calibration.tutorials.tutorials_helpers import helper_data_loader
from pyccapt.calibration.tutorials.tutorials_helpers import helper_temporal_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_special_crop
from pyccapt.calibration.tutorials.tutorials_helpers import helper_t_0_tune
from pyccapt.calibration.tutorials.tutorials_helpers import helper_mc_plot
from pyccapt.calibration.tutorials.tutorials_helpers import helper_3d_reconstruction
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_selection
from pyccapt.calibration.tutorials.tutorials_helpers import helper_visualization
from pyccapt.calibration.tutorials.tutorials_helpers import helper_ion_list

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
In case of recieving the error about pytable library, you have to install the pytables library with conda command. to do that you can open a new cell and copy the line below in it. Then just run it like other cells. The pytables library will be innstalled.
    
`!conda install --yes --prefix {sys.prefix} pytables`
</div>


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
By clicking on the button below, you can select the dataset file you want to crop. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV. The cropped data will be saved in the same directory as the original dataset file in a new directory nammed load_crop. The name of the cropped dataset file will be the same as the original dataset file. The figures will be saved in the same directory as the dataset file.
</div>

In [ ]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## ROI Selection and Data Cropping
    

</div>



<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

    
From the dropdown lists below, you can select the instrument specifications of the dataset. The instrument specifications are the same as the ones used for the calibration process. Data mode is specify the dataset structure. The dataset can be in raw or calibrated mode. The flight path length is the distance between the sample and the detector. The t0 is the time of flight of the ions with the lowest mass-to-charge ratio. The maximum mass-to-charge ratio is the maximum mass-to-charge ratio of tat you want to plot. You can also change it in te related cells. The detector diameter is the diameter of the detector.
</div>


In [ ]:
# create an object for selection of instrument specifications of the dataset
tdc, pulse_mode, flight_path_length, t0, max_mc, det_diam = wd.dataset_instrument_specification_selection()

# Display lists and comboboxes to selected instrument specifications
display(tdc, pulse_mode, flight_path_length, t0, max_mc)

In [ ]:
variables = helper_data_loader.load_data(dataset_path, max_mc.value, flight_path_length.value, pulse_mode.value, tdc.value)
display(variables.data)
display(variables.range_data)

In [ ]:
#load data, if it exists,
try:
    if os.path.exists(variables.result_data_path + '//' + variables.result_data_name + '.h5'):
        variables.data = data_tools.load_data(variables.result_data_path + '//' + variables.result_data_name + '.h5', tdc='pyccapt', mode='processed')
        # exctract needed data from Pandas data frame as an numpy array
        data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
        print('Load data successfully')
    elif os.path.exists(variables.result_data_path + '//' + variables.result_data_name + 'temp' + '.h5'):
        variables.data = data_tools.load_data(variables.result_data_path + '//' + variables.result_data_name + '.h5', tdc='pyccapt', mode='processed')
        # exctract needed data from Pandas data frame as an numpy array
        data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
        print('Continue from the point based on the loaded data')
        
    else:
        print('No data avaliable')
    if os.path.exists(variables.result_data_path + '/' + 'range_' + variables.dataset_name+ '.h5'):
        variables.range_data = data_tools.read_hdf5_through_pandas(variables.result_data_path + '/' + 'range_' + variables.dataset_name+ '.h5')
        # exctract needed data from Pandas data frame as an numpy array
        data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
        print('Load range file')
    else:
        print('No range data avaliable')
    # exctract needed data from Pandas data frame as a numpy array
    data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
    display(variables.data)
    display(variables.range_data)
except Exception as e:
    pass
    # print(e)

<div style="margin: 0 auto; padding: 20px;">

Load a pre saved range file if you want from cell below
</div>


In [ ]:
button_r = widgets.Button(
    description='load range dataset',
)
@button_r.on_click
def open_file_on_click_r(b):
    """
    Event handler for button click event.
    Prompts the user to select a range file and stores the selected file path in the global variable range_path.
    """
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button_r

In [ ]:
if 'range_path' in globals():
    variables.range_data = data_tools.read_hdf5_through_pandas(range_path)
display(variables.range_data)

<div style="margin: 0 auto; padding: 20px;">

# Temporal crop

Select the data by drawing a rectangle over the experiment history. Experiment history is a 2D histogram of the time of flight of the ions versus sequence of evaporation. The experiment history is plotted by clicking on the button below te cell.
</div>


In [ ]:
helper_temporal_crop.call_plot_crop_experiment(variables, pulse_mode.value)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

# Spacial crop

Select the region of maximum concentration of Ions in the below plotted graph to utilize relevant data. To crop you can draw a circle over the filed desorption map. The field desorption map is a 2D histogram of the time of flight of the ions versus the position of the ions on the detector. The field desorption map is plotted by clicking on the button below the cell.
</div>

In [ ]:
helper_special_crop.call_plot_crop_fdm(variables)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Calculate pulses since the last event pulse and ions per pulse. The percentage of loss in ROI selection process will be printed.
</div>

In [ ]:
pulse_pi, ion_pp = data_loadcrop.calculate_ppi_and_ipp(variables.data)

# add two calculated array to the croped dataset
variables.data['pulse_pi'] = pulse_pi.astype(np.uintc)
variables.data['ion_pp'] = ion_pp.astype(np.uintc)

# exctract needed data from Pandas data frame as an numpy array
variables.dld_high_voltage = variables.data['high_voltage (V)'].to_numpy()
variables.dld_pulse = variables.data['pulse'].to_numpy()
variables.dld_t = variables.data['t (ns)'].to_numpy()
variables.dld_x_det = variables.data['x_det (cm)'].to_numpy()
variables.dld_y_det = variables.data['y_det (cm)'].to_numpy()

# save the cropped data
print('tof Crop Loss {:.2f} %'.format((100 - (len(variables.data) / len(variables.data)) * 100)))
#percentage of double event per pulse
print('percentage of double event per pulse', len(ion_pp[ion_pp != 1]) / float(len(ion_pp)))

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In the next cell by changing the t<sub>0</sub> value you can correct the position of H<sub>1</sub> or any other known peak. this correction would be helpful for the position of the peaks in the m/c calibration process.
</div>

In [ ]:
helper_t_0_tune.call_fine_tune_t_0(variables, flight_path_length, pulse_mode, t0)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Remove the data with m/c greater than max m/c and x, y, t = 0. Also add the needed colums for calibration. The data types of the final cropped dataset is displayed below.

</div>

In [ ]:
# add the columns in the dataset for x, y, z, mc, mc calibrated, and t calibrated
helper_data_loader.add_columns(variables, max_mc)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False, temp=True)
display(variables.data)
display(variables.data.dtypes)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Time-of-Flight/Mass-to-charge Calibration
</div>

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
Below you can plot the ToF histogram of the dataset. You can select the peak range of the data you want to plot by drawing a rectangle over peak with holding left click. After that you should apply the voltage and bowl calibration. These three steps should be repeated until you see no improvement in the peak resolution. You can also save the calibration by clicking on the save button. The saved calibration will be used for the next steps.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)
calibration_mode = widgets.Dropdown(
    options=[('time_of_flight', 'tof'), ('mass_to_charge', 'mc')],
    description='calibration mode:')
display(calibration_mode)

In [ ]:
helper_calibration.call_voltage_bowl_calibration(variables, det_diam, calibration_mode)

In [ ]:
variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)
variables.mc_calib_backup = np.copy(variables.mc_calib)

In [ ]:
helper_ion_list.call_ion_list(variables, selector='peak', calibration_mode=calibration_mode)

In [ ]:
# from pyccapt.calibration.mc.tof_tools import mc2tof
# mc2tof(1, 4000, 0, 0, 110)

In [ ]:
helper_mc_plot.call_mc_plot(variables, selector='None')

In [ ]:
variables.data['mc_c (Da)'] = variables.mc_calib
variables.data['t_c (ns)'] = variables.dld_t_calib
# Remove negative mc
threshold = 0
mc_t = variables.data['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
variables.data.drop(np.where(mc_t_mask)[0], inplace=True)
variables.data.reset_index(inplace=True, drop=True)
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False, temp=True)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## 3D Reconstruction
After bowl and voltage calibration we are ready to calculate the 3d reconstruction. In this workflow we calculate the reconstructed x,y,z and then plot the 3d, heatmap, projection plots and mass-to-charge histogram.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

You have to select the main element in your sample from the from dropdown below.
</div>

In [ ]:
element_selected = wd.density_field_selection()
display(element_selected)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

In case that yopu face error about plotly library, like javascripts error. You have check your jupyter lab version is compatibale with the plotly extenstion. Sometimes running `jupyter lab build` command fix the proble.
</div>

In [ ]:
helper_3d_reconstruction.call_x_y_z_calculation(variables, flight_path_length, element_selected)

In [ ]:
variables.plotly_3d_reconstruction

In [ ]:
variables.data['x (nm)'] = variables.x
variables.data['y (nm)'] = variables.y
variables.data['z (nm)'] = variables.z
# save data temporarily
data_tools.save_data(variables.data, variables, hdf=True, epos=False, pos=False, ato_6v=False, csv=False, temp=True)

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

This tutorial outlines a comprehensive workflow for ion selection and organization. Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both figures and data in CSV and HDF5 formats.

</div>

In [ ]:
# exctract needed data from Pandas data frame as an numpy array
data_tools.extract_data(variables.data, variables, flight_path_length.value, max_mc.value)

display(variables.data)

In [ ]:
helper_ion_selection.call_ion_selection(variables)

In [ ]:
variables.range_data

In [ ]:
variables.range_data.dtypes

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the range in the hdf5 and csv format.

</div>

In [ ]:
# save the new data
name_save_file = variables.result_data_path + '/' + 'range_' + variables.dataset_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)
# save data in csv format
name_save_file = variables.result_data_path + '/' + 'range_' + variables.dataset_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Save the cropped dataset. You can specify te output format from list below. The output formats are HDF5, EPOS, POS, ATO, and CSV. The output file will be saved in the same directory as the original dataset file in a new directory nammed load_crop.
</div>

In [ ]:
interact_manual(data_tools.save_data, data=fixed(variables.data), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]),
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]),
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]),
               temp=fixed(False));

-------------------------------------------------------
-------------------------------------------------------

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
    
# Visualization
    
</div>

In [ ]:
variables.data

In [ ]:
helper_visualization.call_visualization(variables)

In [ ]:
variables.plotly_3d_reconstruction

In [ ]:
from IPython.display import display, HTML
display(HTML(variables.animation_detector_html))

In [ ]:
np.subtract.outer([1,2,3], [1,2,3])

In [ ]:
import matplotlib.pyplot as plt
def calculate_sdm(xx, yy, zz, reference_point=(5,5,5), deltax=1, deltay=1, deltaz=1):

    xx = xx.astype(np.float32)
    yy = yy.astype(np.float32)
    zz = zz.astype(np.float32)
    # Clip relative positions to the cell dimensions
    mask_xx = (xx >= reference_point[0] - deltax) & (xx <=  reference_point[0] + deltax)
    mask_yy = (yy >= reference_point[1] - deltay) & (yy <=  reference_point[1] + deltay)
    mask_zz = (zz >= reference_point[2] - deltaz) & (zz <=  reference_point[2] + deltaz)
    mask = mask_xx & mask_yy & mask_zz
    xx = xx[mask]
    yy = yy[mask]
    zz = zz[mask]

    # Calculate relative positions
    dx = np.subtract.outer(xx, xx)
    dy = np.subtract.outer(yy, yy)
    dz = np.subtract.outer(zz, zz)
    
    # Shift to the reference point
    dx = dx + reference_point[0]
    dy = dy + reference_point[1]
    dz = dz + reference_point[2]
    
    return dx, dy, dz


dx, dy, dz = calculate_sdm(variables.x, variables.y, variables.z)

print(dx.shape)
# Create 1D histograms for each axis
histo_dx, bins_dx = np.histogram(dx.flatten(), bins=512)
histo_dy, bins_dy = np.histogram(dy.flatten(), bins=512)
histo_dz, bins_dz = np.histogram(dz.flatten(), bins=512)


# Plot histograms
fig, ax = plt.subplots()
# Plot the histogram
plt.hist(dx.flatten(), bins=250)
ax.set_xlabel('x (nm)')
ax.set_ylabel('Frequency')
# plt.plot(bins_dx[:-1], histo_dx, label='dx')
plt.show()
fig, ax = plt.subplots()
ax.set_xlabel('y (nm)')
ax.set_ylabel('Frequency')
plt.plot(bins_dy[:-1], histo_dy, label='dy')
plt.show()
fig, ax = plt.subplots()
ax.set_xlabel('z (nm)')
ax.set_ylabel('Frequency')
plt.plot(bins_dz[:-1], histo_dz, label='dz')
plt.show()

# 2D histogram for dx and dz
fig, ax = plt.subplots()
ax.set_xlabel('x (nm)')
ax.set_ylabel('z (nm)')
bins = 0.5
# Check if the bin is a tuple
if isinstance(bins, tuple):
    pass
else:
    x_edges = np.arange(dx.min(), dx.max(), bins)
    z_edges = np.arange(dz.min(), dz.max(), bins)
    bins = [x_edges, z_edges]
h = ax.hist2d(dx.flatten(), dz.flatten(), bins=(250,250), cmap='viridis')
plt.colorbar(h[3], ax=ax, label='Frequency')

# plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

def radial_distribution_function(x, y, z, reference_point, dr=0.1, r_max=None):
    # Create a cKDTree for efficient neighbor search
    coordinates = np.column_stack([x, y, z])
    tree = cKDTree(coordinates)

    # Determine the maximum distance for RDF calculation
    if r_max is None:
        r_max = min(np.max(x) - np.min(x), np.max(y) - np.min(y), np.max(z) - np.min(z)) / 2.0

    # Initialize variables for RDF calculation
    bins = np.arange(0, r_max, dr)
    rdf_values = np.zeros(len(bins) - 1)

    # Query neighbors within the distance range of the reference point
    neighbors = tree.query_ball_point(reference_point, r_max)

    # Calculate distances to neighbors
    distances = np.linalg.norm(coordinates[neighbors] - reference_point, axis=1)

    # Update RDF values
    rdf_values += np.histogram(distances, bins=bins)[0]

    # Normalize RDF values
    r = (bins[:-1] + bins[1:]) / 2.0
    volume_shell = 4/3 * np.pi * (bins[1:]**3 - bins[:-1]**3)
    rdf_values /= len(coordinates) * volume_shell

    return r, rdf_values


reference_point = np.array([0, 0, 0])
dr = 1.0  # 1nm

r, rdf = radial_distribution_function(variables.x, variables.y, variables.z, reference_point, dr=dr)

# Plot RDF
fig, ax = plt.subplots()
plt.plot(r, rdf)
plt.xlabel('Distance (nm)')
plt.ylabel('Radial Distribution Function (RDF)')
plt.title(f'RDF for Cell Center at {reference_point}')
plt.show()

In [ ]:
from numba import jit

x = variables.x[0:10000]
y = variables.y[0:10000]
z = variables.z[0:10000]

def calculate_sdm_numba(xx, yy, zz, cell_center=None, deltax=6, deltay=6, deltaz=6):

    if cell_center is not None:
    # Clip relative positions to the cell dimensions
        mask_xx = (xx >= cell_center[0] - deltax) & (xx <=  cell_center[0] + deltax)
        mask_yy = (yy >= cell_center[1] - deltay) & (yy <=  cell_center[1] + deltay)
        mask_zz = (zz >= cell_center[2] - deltaz) & (zz <=  cell_center[2] + deltaz)
        mask = mask_xx & mask_yy & mask_zz
        xx = xx[mask]
        yy = yy[mask]
        zz = zz[mask]
        
    @jit(nopython=True)
    def calculate_sdm(xx, yy, zz):
        xx = xx.astype(np.float32)
        yy = yy.astype(np.float32)
        zz = zz.astype(np.float32)

        n = len(xx)
        dx = np.zeros((n, n), dtype=np.float32)
        dy = np.zeros((n, n), dtype=np.float32)
        dz = np.zeros((n, n), dtype=np.float32)

        for i in range(n):
            for j in range(n):
                dx[i, j] = xx[i] - xx[j]
                dy[i, j] = yy[i] - yy[j]
                dz[i, j] = zz[i] - zz[j]

        return dx, dy, dz
    
    dx, dy, dz = calculate_sdm(xx, yy, zz)
    return dx, dy, dz



def calculate_sdm(xx, yy, zz, cell_center=None, deltax=6, deltay=6, deltaz=6):

    xx = xx.astype(np.float32)
    yy = yy.astype(np.float32)
    zz = zz.astype(np.float32)
    if cell_center is not None:
    # Clip relative positions to the cell dimensions
        mask_xx = (xx >= cell_center[0] - deltax) & (xx <=  cell_center[0] + deltax)
        mask_yy = (yy >= cell_center[1] - deltay) & (yy <=  cell_center[1] + deltay)
        mask_zz = (zz >= cell_center[2] - deltaz) & (zz <=  cell_center[2] + deltaz)
        mask = mask_xx & mask_yy & mask_zz
        xx = xx[mask]
        yy = yy[mask]
        zz = zz[mask]

    # Calculate relative positions
    dx = np.subtract.outer(xx, xx)
    dy = np.subtract.outer(yy, yy)
    dz = np.subtract.outer(zz, zz)
    
    return dx, dy, dz

# Example usage:
cell_center = (5.0, 5.0, 5.0) 


dx, dy, dz = calculate_sdm_numba(x, y, z, cell_center=None)



# Create 1D histograms for each axis
histo_dx, bins_dx = np.histogram(dx.flatten(), bins=124)
histo_dy, bins_dy = np.histogram(dy.flatten(), bins=124)
histo_dz, bins_dz = np.histogram(dz.flatten(), bins=124)

# Plot histograms
fig, ax = plt.subplots()
ax.set_xlabel('dx (Å)')
ax.set_ylabel('Intensity')
plt.plot(bins_dx[:-1], histo_dx, label='dx')
plt.show()
fig, ax = plt.subplots()
ax.set_xlabel('dy (Å)')
ax.set_ylabel('Intensity')
plt.plot(bins_dy[:-1], histo_dy, label='dy')
plt.show()
fig, ax = plt.subplots()
ax.set_xlabel('dz (Å)')
ax.set_ylabel('Intensity')
plt.plot(bins_dz[:-1], histo_dz, label='dz')
plt.show()

# 2D histogram for dx and dz
fig, ax = plt.subplots()
ax.set_xlabel('dx (Å)')
ax.set_ylabel('dz (Å)')
bins = 0.5
# Check if the bin is a tuple
if isinstance(bins, tuple):
    pass
else:
    x_edges = np.arange(dx.min(), dx.max(), bins)
    z_edges = np.arange(dz.min(), dz.max(), bins)
    bins = [x_edges, z_edges]
h = ax.hist2d(dx.flatten(), dz.flatten(), bins=bins, cmap='viridis')
plt.colorbar(h[3], ax=ax, label='Frequency')

# plt.legend()
plt.show()

In [ ]:
from pymatgen.io.cif import CifParser
from pymatgen.core.structure import Structure

# Replace 'your_file.cif' with the actual path to your CIF file
cif_file_path = 'Al.cif'

# Parse the CIF file
parser = CifParser(cif_file_path)
structure = parser.get_structures()[0]

print(structure)
# Make a supercell
structure.make_supercell([6, 6, 6])

# Get atomic positions
positions = structure.as_dataframe()

x_positions = positions['x']
y_positions = positions['y']
z_positions = positions['z']
# Plot the expanded structure using Matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')



# Plot atomic positions
ax.scatter(x_positions, y_positions, z_positions, c='b', marker='o', s=100)

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Show the plot
plt.show()


In [ ]:
# Replace 'your_file.xyz' with the actual path to your XYZ file
file_path = 'Al_big.xyz'

# Read the XYZ file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Extract the number of atoms
num_atoms = int(lines[0])

# Initialize arrays to store x, y, and z coordinates
x_positions = np.zeros(num_atoms)
y_positions = np.zeros(num_atoms)
z_positions = np.zeros(num_atoms)

# Loop through lines starting from the second line (skipping the number of atoms line)
for i, line in enumerate(lines[2:]):
    parts = line.split()
    x_positions[i] = float(parts[1])
    y_positions[i] = float(parts[2])
    z_positions[i] = float(parts[3])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x_positions, y_positions, z_positions, s=50, c='b', marker='o', edgecolors='k')

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('Atomic Structure')

plt.show()

# Define the center and radius of the half sphere
sphere_center = np.array([x_positions.mean(), y_positions.mean(), z_positions.mean()])
sphere_radius = 10.0  # Adjust the radius as needed

# Create a boolean mask to filter atoms outside the half sphere
distance_from_center = np.linalg.norm(np.column_stack((x_positions, y_positions, z_positions)) - sphere_center, axis=1)
atoms_in_half_sphere = distance_from_center <= sphere_radius

# Apply the mask to filter out atoms outside the half sphere
x_positions_half_sphere = x_positions[atoms_in_half_sphere]
y_positions_half_sphere = y_positions[atoms_in_half_sphere]
z_positions_half_sphere = z_positions[atoms_in_half_sphere]

# Plot the remaining atoms in the half sphere
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x_positions_half_sphere, y_positions_half_sphere, z_positions_half_sphere, s=50, c='b', marker='o', edgecolors='k')

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('Atomic Structure - Half Sphere')

plt.show()

sample_2d_histogram(x_positions_half_sphere, z_positions_half_sphere, bins=(512, 512), xlabel='x (nm)', ylabel='z (nm)')

In [ ]:
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize

####################
# # Create the aluminum bulk crystal structure
# aluminum = bulk('Al', 'fcc', a=4.49580, b=4.49580, c=8.99160, alpha=90.0,  cubic=True)

# # Replicate the unit cell in x, y, and z directions to create a larger volume
# aluminum = aluminum.repeat((10, 10, 20))

# # Get the atomic positions
# positions = aluminum.get_positions()

# # Extract x, y, z coordinates from the positions
# x, y, z = positions[:, 0], positions[:, 1], positions[:, 2]

#################
# Create a small unit cell of aluminum
aluminum_unit_cell = bulk('Al', 'fcc', a=4.49580, b=4.49580, c=8.99160, alpha=90.0,  cubic=True)
view(aluminum_unit_cell, viewer='x3d', show_unit_cell=True)

# Calculate the dimensions of the supercell
supercell_dimensions = (5, 5, 10)
cell_parameters = cell_to_cellpar(aluminum_unit_cell.cell)
supercell_parameters = [dim * param for dim, param in zip(supercell_dimensions, cell_parameters)]

# Create a supercell
aluminum = Atoms(cell=supercell_parameters)
aluminum.extend(aluminum_unit_cell.repeat(supercell_dimensions))


# Get atomic positions
positions = aluminum.get_positions()

# Separate x, y, and z coordinates into arrays
x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]
######################


# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o', s=20, label='Aluminum Atoms')

# Set labels
ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Z (Å)')
ax.set_title('3D Crystal Structure of Aluminum')

# Show the plot
plt.show()



In [ ]:
# Function to calculate 3D FFT and plot the result
def calculate_3d_fft_and_plot(x, y, z):
    # Combine x, y, z coordinates into a complex 3D array
    data = x + 1j * y + 1j**2 * z

    # Perform 3D FFT
    fft_result = np.fft.fftn(data)

    # Get the magnitudes of the FFT result
    fft_magnitude = np.abs(fft_result)

    # Plot the 3D FFT magnitudes
    fig_fft = plt.figure()
    ax_fft = fig_fft.add_subplot(111, projection='3d')
    
    # Get the coordinates of non-zero values in fft_magnitude
    nonzero_coords = np.where(fft_magnitude > 1e-3)
    
    # Scatter plot with x, y, z coordinates
    ax_fft.scatter(nonzero_coords[0], nonzero_coords[1], nonzero_coords[2], c='r', marker='o', s=20, label='FFT Peaks')
    ax_fft.set_xlabel('Frequency X (1/Å)')
    ax_fft.set_ylabel('Frequency Y (1/Å)')
    ax_fft.set_zlabel('Frequency Z (1/Å)')
    ax_fft.set_title('3D FFT of Crystal Structure')

    plt.show()

# Calculate 3D FFT for x, y, and z coordinates
calculate_3d_fft_and_plot(x, y, z)